In [1]:
library(forecast)
library(M4comp2018)
library(dplyr)

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
etsl=load('week_ets_datalist.RData')
ets_datalist <- eval(parse(text = etsl))
thetafl=load('week_thetaf_datalist.RData')
thetaf_datalist <- eval(parse(text = thetafl))

In [3]:
ets_predh=ets_datalist[[2]]
thetaf_predh=thetaf_datalist[[2]]

In [4]:
ets_pred_res=ets_datalist[[1]]
thetaf_pred_res=thetaf_datalist[[1]]

In [5]:
etsoptl=load('weekly_ets_opt_pre_result.RData')
ets_opt_pre_list<- eval(parse(text = etsoptl))
thetafoptl=load('weekly_thetaf_opt_pre_result.RData')
thetaf_opt_pre_list<- eval(parse(text = thetafoptl))

In [6]:
time_matrix <- matrix(0,ncol = 3, nrow =3)
colnames(time_matrix) <- c("user_time", "system_time", "elapsed_time")
start_time = Sys.time()

In [7]:
ets_pr_min=ets_opt_pre_list[[1]]
ets_pr_mean=ets_opt_pre_list[[2]]
thetaf_pr_min=thetaf_opt_pre_list[[1]]
thetaf_pr_mean=thetaf_opt_pre_list[[2]]

In [8]:
head(ets_pr_min)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
4,4.0002679825,4,3.88148185
4,3.9983553886,0,4.05099395
4,4.0006723404,1,4.06134082
1,0.9991121888,3,0.86301980
0,-0.0005784677,4,0.05222995
0,-0.0002692383,3,0.03920312


In [9]:
ets_pr_min=ets_pr_min+1
ets_pr_mean=ets_pr_mean+1
thetaf_pr_min=thetaf_pr_min+1
thetaf_pr_mean=thetaf_pr_mean+1


In [10]:
head(ets_pr_min)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
5,5.0002680,5,4.881482
5,4.9983554,1,5.050994
5,5.0006723,2,5.061341
2,1.9991122,4,1.863020
1,0.9994215,5,1.052230
1,0.9997308,4,1.039203


In [11]:
round5=function(x)
    {
    l=round(x)
    for(i in 1:length(l))
    {
    if(l[i]>5)
        {
        l[i]=5
    }
    if(l[i]<1)
    {
        l[i]=1
    }
}
    return(l)
}

In [12]:
ets_pr_min[,2]=round5(ets_pr_min[,2])
ets_pr_min[,4]=round5(ets_pr_min[,4])

In [13]:
ets_pr_mean[,2]=round5(ets_pr_mean[,2])
ets_pr_mean[,4]=round5(ets_pr_mean[,4])

In [14]:
thetaf_pr_min[,2]=round5(thetaf_pr_min[,2])
thetaf_pr_min[,4]=round5(thetaf_pr_min[,4])

In [15]:
thetaf_pr_mean[,2]=round5(thetaf_pr_mean[,2])
thetaf_pr_mean[,4]=round5(thetaf_pr_mean[,4])

In [16]:
head(ets_pr_min)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
5,5,5,5
5,5,1,5
5,5,2,5
2,2,4,2
1,1,5,1
1,1,4,1


In [17]:
Weekly_M4 <- Filter(function(l) l$period == "Weekly", M4)

In [18]:
data=Weekly_M4

In [19]:
datalength=length(data)
h=data[[1]]$h
m=5
n=4
freq=frequency(data[[1]]$x)

In [20]:
realxx=matrix(0,datalength,h)
for(i in 1:datalength){
    realxx[i,]=data[[i]]$xx
}

In [21]:
length5=matrix(0,datalength,m)
for(k in 1:datalength)
{
y = data[[k]]$x
y_l=length(y)
loc = 1:length(y)
loc_m = as.integer(loc*m/length(y))
filt_d = data.frame(y,loc_m)
filt_0=filter(filt_d ,loc_m==0)
m_l=count(filt_0)
m_l=as.integer(m_l)
length5[k,]=c(0,1*m_l,2*m_l,3*m_l,4*m_l)
    }

In [22]:
end_time = Sys.time()
time_matrix[1,]=end_time-start_time

In [23]:
end_time-start_time

Time difference of 13.04328 secs

## ETS

In [24]:
start_time = Sys.time()

In [25]:
set.seed(100)
index = sample(2,length(data),replace = TRUE,prob=c(0.7,0.3))
testindex=c(1:length(data))[index==2]

In [26]:
meanpre=function(pred_array,opt_pr,h,testindex)
    {
    data_length=length(testindex)
    meanpreh=matrix(0,data_length,h)
    for (i in 1:data_length){
    index=testindex[i]
    loc=opt_pr[index]
    predh=pred_array[index,loc,,]
    predh=unique(predh)
    meanpreh[i,]=apply(predh,2,mean)
}
    return(meanpreh)
}

In [27]:
predres=function(pred_matrix,data,opt_pr,lengthmatrix,h,testindex)
    {
    data_length=length(testindex)
    pred_res=matrix(0,data_length,6)
    model=ets(data[[1]]$x)
    fore_l=forecast(model,h=h)
    freq=frequency(data[[1]]$x)
    for (i in 1:data_length){
    index=testindex[i]
    loc=opt_pr[index]
    start=lengthmatrix[i,loc]
    y_all=data[[index]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[index]]$xx
    fore_l$mean=ts(pred_matrix[i,],start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    pred_res[i,]=res[2,1:6]
    }
    return(pred_res)
    }

In [28]:
predres_mul=function(pred_matrix,data,opt_pr_mul,lengthmatrix,h,testindex)
    {
    data_length=length(testindex)
    pred_res=matrix(0,data_length,6)
    model=ets(data[[1]]$x)
    fore_l=forecast(model,h=h)
    freq=frequency(data[[1]]$x)
    for (i in 1:data_length){
    index=testindex[i]
    loc=min(opt_pr_mul[index,])
    start=lengthmatrix[i,loc]
    y_all=data[[index]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[index]]$xx
    fore_l$mean=ts(pred_matrix[i,],start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    pred_res[i,]=res[2,1:6]
    }
    return(pred_res)
    }

In [29]:
etsxgbcls_min=meanpre(ets_predh,ets_pr_min[,1],h,testindex)
etsxgbreg_min=meanpre(ets_predh,ets_pr_min[,2],h,testindex)
etslgbcls_min=meanpre(ets_predh,ets_pr_min[,3],h,testindex)
etslgbreg_min=meanpre(ets_predh,ets_pr_min[,4],h,testindex)

In [30]:
etsxgbcls_mean=meanpre(ets_predh,ets_pr_mean[,1],h,testindex)
etsxgbreg_mean=meanpre(ets_predh,ets_pr_mean[,2],h,testindex)
etslgbcls_mean=meanpre(ets_predh,ets_pr_mean[,3],h,testindex)
etslgbreg_mean=meanpre(ets_predh,ets_pr_mean[,4],h,testindex)

### The error of each improved prediction result

In [31]:
lengthtest=length5[testindex,]

In [32]:
etsminxgbclsres=predres(etsxgbcls_min,data,ets_pr_min[,1],lengthtest,h,testindex)
etsminxgbregres=predres(etsxgbreg_min,data,ets_pr_min[,2],lengthtest,h,testindex)
etsminlgbclsres=predres(etslgbcls_min,data,ets_pr_min[,3],lengthtest,h,testindex)
etsminlgbregres=predres(etslgbreg_min,data,ets_pr_min[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

In [33]:
etsmeanxgbclsres=predres(etsxgbcls_mean,data,ets_pr_mean[,1],lengthtest,h,testindex)
etsmeanxgbregres=predres(etsxgbreg_mean,data,ets_pr_mean[,2],lengthtest,h,testindex)
etsmeanlgbclsres=predres(etslgbcls_mean,data,ets_pr_mean[,3],lengthtest,h,testindex)
etsmeanlgbregres=predres(etslgbreg_mean,data,ets_pr_mean[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

In [34]:
end_time = Sys.time()
time_matrix[2,]=end_time-start_time

### The results of the two methods of determining labels are averaged respectively

In [35]:
etsallmin=(etsxgbcls_min+etslgbcls_min+etsxgbreg_min+etslgbreg_min)/4
etsallmean=(etsxgbcls_mean+etslgbcls_mean+etsxgbreg_mean+etslgbreg_mean)/4

In [36]:
etsallminres=predres_mul(etsallmin,data,ets_pr_min,lengthtest,h,testindex)
etsallmeanres=predres_mul(etsallmean,data,ets_pr_mean,lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”


### Under the above classification, the improved forecast results of the classification and regression models are averaged respectively

In [37]:
etsclsmin=(etsxgbcls_min+etslgbcls_min)/2
etsregmin=(etsxgbreg_min+etslgbreg_min)/2
etsclsmean=(etsxgbcls_mean+etslgbcls_mean)/2
etsregmean=(etsxgbreg_mean+etslgbreg_mean)/2

In [38]:
etsclsminres=predres_mul(etsclsmin,data,ets_pr_min[,c(1,3)],lengthtest,h,testindex)
etsregminres=predres_mul(etsregmin,data,ets_pr_min[,c(2,4)],lengthtest,h,testindex)
etsclsmeanres=predres_mul(etsclsmean,data,ets_pr_mean[,c(1,3)],lengthtest,h,testindex)
etsregmeanres=predres_mul(etsregmean,data,ets_pr_mean[,c(2,4)],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

### Random slice

In [39]:
etspred_change<- aperm(ets_predh[1:datalength,1:m,1:n,1:h], c(1, 3, 2, 4))  
# 使用array函数将其转换为新的形状
etspred_change <- array(etspred_change, dim = c(datalength, m*n, h))

In [40]:
model=ets(data[[1]]$x)
fore_l=forecast(model,h=h)
etsrandomres=matrix(0,datalength,6)
for (i in 1:datalength){
    random=sample(m*n,n)
    minloc=min(random)
    # 使用cut函数将数据分割为区间
    loc <- cut(minloc, breaks =seq(0,m*n,n), labels = FALSE)
    loc<-as.numeric(loc)
    etsrandompred=apply(etspred_change[i,random,],2,mean)
    start=length5[i,loc]
    y_all=data[[i]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[i]]$xx
    fore_l$mean=ts(etsrandompred,start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    etsrandomres[i,]=res[2,1:6]
    }

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”


In [41]:
mean1=function(x)
    {
# 使用is.finite函数检查哪些值是有限的
finite_values <- x[is.finite(x)]
# 计算有限值的平均值
d=mean(finite_values, na.rm = TRUE)
return(d)
}

In [42]:
print(apply(ets_pred_res[testindex,1,1,],2,mean1))
print(apply(etsminxgbclsres,2,mean1))
print(apply(etsminxgbregres,2,mean1))
print(apply(etsminlgbclsres,2,mean1))
print(apply(etsminlgbregres,2,mean1))

[1] 129.295832 460.092114 371.571332   2.381339   9.169361   2.496023
[1] 124.148027 463.258463 371.811607   2.202851   9.207675   1.973157
[1] 124.899756 470.309816 378.935278   2.137638   9.303549   2.068130
[1] 130.216449 466.719550 374.027590   2.295915   9.146254   2.337815
[1] 133.648690 462.990166 372.725217   2.463288   9.247754   2.006501


In [43]:
print(apply(etsmeanxgbclsres,2,mean1))
print(apply(etsmeanxgbregres,2,mean1))
print(apply(etsmeanlgbclsres,2,mean1))
print(apply(etsmeanlgbregres,2,mean1))

[1] 124.855523 457.534760 366.858777   2.085826   9.111625   1.963596
[1] 120.528153 462.570741 370.407452   2.061137   9.183061   1.982919
[1] 123.255360 464.671631 370.658094   2.260790   9.150482   2.354584
[1] 123.044309 462.931739 371.249449   2.096142   9.238237   2.070775


In [44]:
print(apply(etsallminres,2,mean1))
print(apply(etsallmeanres,2,mean1))

[1] 128.228230 463.264691 372.402838   2.274923   9.175525   2.249332
[1] 122.920837 459.828033 368.253844   2.125974   9.146430   2.285263


In [45]:
print(apply(etsclsminres,2,mean1))
print(apply(etsregminres,2,mean1))
print(apply(etsclsmeanres,2,mean1))
print(apply(etsregmeanres,2,mean1))

[1] 127.182238 464.010519 372.393826   2.249383   9.162724   2.256150
[1] 129.274223 465.381169 374.825902   2.300463   9.254800   2.021180
[1] 124.055442 458.948660 367.189844   2.173308   9.105564   2.280011
[1] 121.786231 462.379350 370.526394   2.078640   9.206530   2.012985


In [46]:
print(apply(etsrandomres[testindex,],2,mean))

[1] 133.353563 464.109761 374.199543   2.411579   9.193610   2.427952


In [47]:
etsall=matrix(0,16,6)
etsall[1,]=apply(ets_pred_res[testindex,1,1,],2,mean1)[1:6]
etsall[2,]=apply(etsminxgbclsres,2,mean1)
etsall[3,]=apply(etsminxgbregres,2,mean1)
etsall[4,]=apply(etsminlgbclsres,2,mean1)
etsall[5,]=apply(etsminlgbregres,2,mean1)
etsall[6,]=apply(etsmeanxgbclsres,2,mean1)
etsall[7,]=apply(etsmeanxgbregres,2,mean1)
etsall[8,]=apply(etsmeanlgbclsres,2,mean1)
etsall[9,]=apply(etsmeanlgbregres,2,mean1)
etsall[10,]=apply(etsallminres,2,mean1)
etsall[11,]=apply(etsallmeanres,2,mean1)
etsall[12,]=apply(etsclsminres,2,mean1)
etsall[13,]=apply(etsregminres,2,mean1)
etsall[14,]=apply(etsclsmeanres,2,mean1)
etsall[15,]=apply(etsregmeanres,2,mean1)
etsall[16,]=apply(etsrandomres[testindex,],2,mean)

In [48]:
etsall

129.2958,460.0921,371.5713,2.381339,9.169361,2.496023
124.1480,463.2585,371.8116,2.202851,9.207675,1.973157
124.8998,470.3098,378.9353,2.137638,9.303549,2.068130
130.2164,466.7196,374.0276,2.295915,9.146254,2.337815
133.6487,462.9902,372.7252,2.463288,9.247754,2.006501
124.8555,457.5348,366.8588,2.085826,9.111625,1.963596
120.5282,462.5707,370.4075,2.061137,9.183061,1.982919
123.2554,464.6716,370.6581,2.260790,9.150482,2.354584
123.0443,462.9317,371.2494,2.096142,9.238237,2.070775
128.2282,463.2647,372.4028,2.274923,9.175525,2.249332
122.9208,459.8280,368.2538,2.125974,9.146430,2.285263


In [49]:
write.csv(etsall,'ets_week_final_res.csv')

## thetaf

### The error of each improved prediction result

In [50]:
start_time = Sys.time()

In [51]:
thetafxgbcls_min=meanpre(thetaf_predh,thetaf_pr_min[,1],h,testindex)
thetafxgbreg_min=meanpre(thetaf_predh,thetaf_pr_min[,2],h,testindex)
thetaflgbcls_min=meanpre(thetaf_predh,thetaf_pr_min[,3],h,testindex)
thetaflgbreg_min=meanpre(thetaf_predh,thetaf_pr_min[,4],h,testindex)

In [52]:
thetafxgbcls_mean=meanpre(thetaf_predh,thetaf_pr_mean[,1],h,testindex)
thetafxgbreg_mean=meanpre(thetaf_predh,thetaf_pr_mean[,2],h,testindex)
thetaflgbcls_mean=meanpre(thetaf_predh,thetaf_pr_mean[,3],h,testindex)
thetaflgbreg_mean=meanpre(thetaf_predh,thetaf_pr_mean[,4],h,testindex)

In [53]:
thetafminxgbclsres=predres(thetafxgbcls_min,data,thetaf_pr_min[,1],lengthtest,h,testindex)
thetafminxgbregres=predres(thetafxgbreg_min,data,thetaf_pr_min[,2],lengthtest,h,testindex)
thetafminlgbclsres=predres(thetaflgbcls_min,data,thetaf_pr_min[,3],lengthtest,h,testindex)
thetafminlgbregres=predres(thetaflgbreg_min,data,thetaf_pr_min[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

In [54]:
thetafmeanxgbclsres=predres(thetafxgbcls_mean,data,thetaf_pr_mean[,1],lengthtest,h,testindex)
thetafmeanxgbregres=predres(thetafxgbreg_mean,data,thetaf_pr_mean[,2],lengthtest,h,testindex)
thetafmeanlgbclsres=predres(thetaflgbcls_mean,data,thetaf_pr_mean[,3],lengthtest,h,testindex)
thetafmeanlgbregres=predres(thetaflgbreg_mean,data,thetaf_pr_mean[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

In [55]:
end_time = Sys.time()
time_matrix[3,]=end_time-start_time

In [73]:
end_time-start_time

Time difference of 4.373411 secs

### The results of the two methods of determining labels are averaged respectively

In [56]:
thetafallmin=(thetafxgbcls_min+thetaflgbcls_min+thetafxgbreg_min+thetaflgbreg_min)/4
thetafallmean=(thetafxgbcls_mean+thetaflgbcls_mean+thetafxgbreg_mean+thetaflgbreg_mean)/4

In [57]:
thetafallminres=predres_mul(thetafallmin,data,thetaf_pr_min,lengthtest,h,testindex)
thetafallmeanres=predres_mul(thetafallmean,data,thetaf_pr_mean,lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”


### Under the above classification, the improved forecast results of the classification and regression models are averaged respectively

In [58]:
thetafclsmin=(thetafxgbcls_min+thetaflgbcls_min)/2
thetafregmin=(thetafxgbreg_min+thetaflgbreg_min)/2
thetafclsmean=(thetafxgbcls_mean+thetaflgbcls_mean)/2
thetafregmean=(thetafxgbreg_mean+thetaflgbreg_mean)/2

In [59]:
thetafclsminres=predres_mul(thetafclsmin,data,thetaf_pr_min[,c(1,3)],lengthtest,h,testindex)
thetafregminres=predres_mul(thetafregmin,data,thetaf_pr_min[,c(2,4)],lengthtest,h,testindex)
thetafclsmeanres=predres_mul(thetafclsmean,data,thetaf_pr_mean[,c(1,3)],lengthtest,h,testindex)
thetafregmeanres=predres_mul(thetafregmean,data,thetaf_pr_mean[,c(2,4)],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

### Random slice

In [60]:
thetafpred_change<- aperm(thetaf_predh[1:datalength,1:m,1:n,1:h], c(1, 3, 2, 4))  
# 使用array函数将其转换为新的形状
thetafpred_change <- array(thetafpred_change, dim = c(datalength, m*n, h))

In [61]:
model=ets(data[[1]]$x)
fore_l=forecast(model,h=h)
thetafrandomres=matrix(0,datalength,6)
for (i in 1:datalength){
    random=sample(m*n,n)
    minloc=min(random)
    # 使用cut函数将数据分割为区间
    loc <- cut(minloc, breaks =seq(0,m*n,n), labels = FALSE)
    loc<-as.numeric(loc)
    thetafrandompred=apply(thetafpred_change[i,random,],2,mean)
    start=length5[i,loc]
    y_all=data[[i]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[i]]$xx
    fore_l$mean=ts(thetafrandompred,start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    thetafrandomres[i,]=res[2,1:6]
    }

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”


In [62]:
print(apply(thetaf_pred_res[testindex,1,1,],2,mean1))
print(apply(thetafminxgbclsres,2,mean1))
print(apply(thetafminxgbregres,2,mean1))
print(apply(thetafminlgbclsres,2,mean1))
print(apply(thetafminlgbregres,2,mean1))

[1] 166.825601 469.369638 375.619970   3.253704   9.048444   2.658670
[1] 160.889080 473.265030 381.147706   3.268522   9.283953   2.105750
[1] 187.973864 481.653656 389.072508   3.814392   9.328815   2.116661
[1] 178.320768 476.139778 382.634731   3.550074   9.211429   2.444831
[1] 192.648632 483.170918 390.149231   4.000789   9.410544   2.123130


In [63]:
print(apply(thetafmeanxgbclsres,2,mean1))
print(apply(thetafmeanxgbregres,2,mean1))
print(apply(thetafmeanlgbclsres,2,mean1))
print(apply(thetafmeanlgbregres,2,mean1))

[1] 168.860984 469.162785 376.250724   3.302417   9.125922   2.078289
[1] 180.347398 476.772867 383.202663   3.605354   9.238592   2.088955
[1] 180.264933 481.102011 387.028909   3.511584   9.255206   2.469855
[1] 181.995490 478.822746 385.062242   3.593962   9.272086   2.144638


In [64]:
print(apply(thetafallminres,2,mean1))
print(apply(thetafallmeanres,2,mean1))

[1] 179.958086 476.790723 384.115979   3.658445   9.267829   2.329888
[1] 177.867201 475.673844 382.176590   3.503329   9.202694   2.309751


In [65]:
print(apply(thetafallminres,2,mean1))
print(apply(thetafallmeanres,2,mean1))

[1] 179.958086 476.790723 384.115979   3.658445   9.267829   2.329888
[1] 177.867201 475.673844 382.176590   3.503329   9.202694   2.309751


In [66]:
print(apply(thetafclsminres,2,mean1))
print(apply(thetafclsmeanres,2,mean1))
print(apply(thetafregminres,2,mean1))
print(apply(thetafregmeanres,2,mean1))

[1] 169.604924 473.537730 380.830333   3.409298   9.219350   2.326297
[1] 174.562959 474.310097 380.958468   3.407001   9.171001   2.291155
[1] 190.311248 481.926070 389.250289   3.907591   9.360845   2.097294
[1] 181.171444 477.600426 383.917650   3.599658   9.248211   2.115278


In [67]:
print(apply(thetafrandomres[testindex,],2,mean1))

[1] 170.997229 475.226119 381.904982   3.457914   9.178614   2.496642


In [68]:
thetafall=matrix(0,16,6)
thetafall[1,]=apply(thetaf_pred_res[testindex,1,1,],2,mean1)[1:6]
thetafall[2,]=apply(thetafminxgbclsres,2,mean1)
thetafall[3,]=apply(thetafminxgbregres,2,mean1)
thetafall[4,]=apply(thetafminlgbclsres,2,mean1)
thetafall[5,]=apply(thetafminlgbregres,2,mean1)
thetafall[6,]=apply(thetafmeanxgbclsres,2,mean1)
thetafall[7,]=apply(thetafmeanxgbregres,2,mean1)
thetafall[8,]=apply(thetafmeanlgbclsres,2,mean1)
thetafall[9,]=apply(thetafmeanlgbregres,2,mean1)
thetafall[10,]=apply(thetafallminres,2,mean1)
thetafall[11,]=apply(thetafallmeanres,2,mean1)
thetafall[12,]=apply(thetafclsminres,2,mean1)
thetafall[13,]=apply(thetafclsmeanres,2,mean1)
thetafall[14,]=apply(thetafregminres,2,mean1)
thetafall[15,]=apply(thetafregmeanres,2,mean1)
thetafall[16,]=apply(thetafrandomres[testindex,],2,mean1)

In [69]:
thetafall

166.8256,469.3696,375.6200,3.253704,9.048444,2.658670
160.8891,473.2650,381.1477,3.268522,9.283953,2.105750
187.9739,481.6537,389.0725,3.814392,9.328815,2.116661
178.3208,476.1398,382.6347,3.550074,9.211429,2.444831
192.6486,483.1709,390.1492,4.000789,9.410544,2.123130
168.8610,469.1628,376.2507,3.302417,9.125922,2.078289
180.3474,476.7729,383.2027,3.605354,9.238592,2.088955
180.2649,481.1020,387.0289,3.511584,9.255206,2.469855
181.9955,478.8227,385.0622,3.593962,9.272086,2.144638
179.9581,476.7907,384.1160,3.658445,9.267829,2.329888
177.8672,475.6738,382.1766,3.503329,9.202694,2.309751


In [74]:
write.csv(thetafall,'thetaf_weekly_final_res.csv')

In [71]:
time_matrix

user_time,system_time,elapsed_time
13.043276,13.043276,13.043276
4.231220,4.231220,4.231220
4.373411,4.373411,4.373411


In [75]:
write.csv(time_matrix,'Weekly_time_matrix.csv')